In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

from dotenv import load_dotenv

# get project root (one up from /jupyter)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# get 'src folder path and add to sys.path -> enables importing of our own pythong modules
src_path = os.path.join(project_root, 'src')
sys.path.append(src_path)

# same for database path
db_path = os.path.join(project_root, 'sqlite_db')

load_dotenv()
SQLITE_DB_NAME = os.getenv("SQLITE_DB_NAME") or "EMBRACE_SQLITE_DB.db"

from sqlite_connector import SQLiteConnector
SQL = SQLiteConnector(f"{db_path}/{SQLITE_DB_NAME}", debug=True)

# grab data from DB
db_agg_sessions = None
db_sessions_col_names = None
sessions_df = None
SQL.connect_db()

if (SQL._cursor and SQL._connection):
    db_sessions_col_names =  list(map(lambda x: x[0], SQL._cursor.execute("SELECT * FROM watch_hist_agg_sessions").description))
    db_agg_sessions = SQL._cursor.fetchall()
    SQL._connection.commit()

if (db_agg_sessions == None or db_sessions_col_names == None):
    print("ERROR db_agg_sessions is None")
    exit()

column_names = {}
for i in range(len(db_sessions_col_names)):
    column_names[i] = db_sessions_col_names[i]
    
print(column_names)
    
sessions_df = pd.DataFrame(db_agg_sessions)
sessions_df.rename(columns=column_names, inplace=True)
# sessions_df.hist(bins=50, figsize=(20,15))
# sessions_df["total_seconds_watched"].hist(bins=50, range=(0, sessions_df["total_seconds_watched"].max()))
# sessions_df["session_span_minutes"].hist(bins=50, range=(0, sessions_df["session_span_minutes"].max()))
# sessions_df["session_count"].hist(bins=50, range=(0, sessions_df["session_count"].max()))
# plt.show()
    
sessions_df.describe(include='all').round(2)



DB connection tested and successful!
{0: 'session_id', 1: 'user_id', 2: 'item_id', 3: 'session_start_timestamp', 4: 'session_end_timestamp', 5: 'session_span_minutes', 6: 'total_seconds_watched', 7: 'session_count', 8: 'completion_ratio', 9: 'outcome', 10: 'created_timestamp'}


,session_id,user_id,item_id,session_start_timestamp,session_end_timestamp,session_span_minutes,total_seconds_watched,session_count,completion_ratio,outcome,created_timestamp
count,764.00,764,764,764,764,764.00,764.00,764.00,760.00,764,764
unique,NaN,5,416,764,764,NaN,NaN,NaN,NaN,5,1
top,NaN,2d62099a3b7842e3a48e8246d6b7c8c6,174237,2025-06-02 21:16:40,2025-06-02 21:16:40,NaN,NaN,NaN,NaN,completed,2025-09-02 09:54:16
freq,NaN,370,8,1,1,NaN,NaN,NaN,NaN,301,764
mean,382.50,NaN,NaN,NaN,NaN,0.93,2193.44,1.38,0.57,NaN,NaN
std,220.69,NaN,NaN,NaN,NaN,3.89,9548.78,1.03,0.39,NaN,NaN
min,1.00,NaN,NaN,NaN,NaN,0.00,0.00,1.00,0.00,NaN,NaN
25%,191.75,NaN,NaN,NaN,NaN,0.00,618.75,1.00,0.18,NaN,NaN
50%,382.50,NaN,NaN,NaN,NaN,0.00,1587.00,1.00,0.60,NaN,NaN
75%,573.25,NaN,NaN,NaN,NaN,0.00,2583.25,1.00,0.98,NaN,NaN


In [ ]:
db_session_lib_items = None
if (SQL._cursor and SQL._connection):
    SQL._cursor.execute("""SELECT
                            a.user_id, 
                            a.item_id, 
                            l.item_name, 
                            g.genre_name 
                        FROM 
                            watch_hist_agg_sessions AS a 
                        INNER JOIN
                            library_items AS l ON a.item_id = l.item_id
                        INNER JOIN
                            item_genres AS g ON l.item_id = g.item_id
                        """).description
    db_session_lib_items = SQL._cursor.fetchall()
    SQL._connection.commit()

# print(db_session_lib_items)
items_df = pd.DataFrame(db_session_lib_items)
items_df.rename(columns={
    0: "user_id",
    1: "Item ID",

})
items_df.head(10)

,0,1,2,3
0,6c5840cf8c1e4d19861b37cc31d36e38,12,10 Things I Hate About You,Comedy
1,6c5840cf8c1e4d19861b37cc31d36e38,12,10 Things I Hate About You,Romance
2,6c5840cf8c1e4d19861b37cc31d36e38,12,10 Things I Hate About You,Drama
3,c00e01aa190942e4b65604f00978083e,2083,Problem Child 3,Comedy
4,c00e01aa190942e4b65604f00978083e,2083,Problem Child 3,Family
5,c00e01aa190942e4b65604f00978083e,2083,Problem Child 3,TV Movie
6,6c5840cf8c1e4d19861b37cc31d36e38,10432,The Instigators,Crime
7,6c5840cf8c1e4d19861b37cc31d36e38,10432,The Instigators,Comedy
8,f49c1281cadb43499181b8759f6ae81b,11627,Bloodsport,Action
9,f49c1281cadb43499181b8759f6ae81b,11627,Bloodsport,Action
